In [ ]:
# Instalação de dependências / Dependency installation
!pip install streamlit --quiet
!npm install localtunnel --global --silent
!pip install openpyxl --quiet

In [ ]:
# Importação de bibliotecas / Library imports
import pandas as pd
import streamlit as st
import os
import numpy as np
import plotly.express as px


In [ ]:
# Carregamento do dataset / Dataset loading
df = pd.read_csv('dados.csv')

In [ ]:
# Inspeção de tipos e dados ausentes / Data types and missing values inspection
df.info()

In [ ]:
# Visualização das primeiras linhas / Preview of the first rows
display(df.head())

In [ ]:
# Padronização dos nomes das colunas / Standardizing column names
colunas_novas = {
    'Safra': 'safra',
    'Regiao': 'regiao',
    'Municipio': 'municipio',
    'Latitude': 'latitude',
    'Longitude': 'longitude',
    'Altitude': 'altitude',
    'Rendimento (Kg/ha)': 'rendimento_kg_ha',
    'Reduçao-chuvas (%)': 'reducao_chuvas_perc',
    'Queda Rendimento 18-19': 'queda_rendimento_perc'
}

df.rename(columns=colunas_novas, inplace=True)

In [ ]:
# Listagem de colunas atualizadas / List of updated columns
display(df.columns.tolist())


In [ ]:
# Definição da função de classificação geográfica / Definition of geographical classification function
def classificar_regiao(latitude):
  if latitude>= -24:
    return 'Norte_Parana'
  elif latitude < -24 and latitude >= -25.5:
    return 'Centro_Parana'
  else:
    return 'Sul_Parana'

In [ ]:
# Aplicação da classificação ao dataframe / Applying classification to the dataframe
df['regiao_geo'] = df['latitude'].apply(classificar_regiao)

In [ ]:
# Análise da distribuição por região / Analysis of distribution by region
print("\n Distribuição da Nova Coluna 'regiao_geo' \n")
df_distribuicao = df['regiao_geo'].value_counts().to_frame().rename_axis('Regiao Geográfica').reset_index()
df_distribuicao.columns = ['Regiao Geográfica', 'N° de Observações']
display(df_distribuicao.style.set_properties(**{'text-align': 'center'}))

In [ ]:
# Seleção de variáveis para estatística descritiva / Selection of variables for descriptive statistics
colunas_chave = ['rendimento_kg_ha', 'reducao_chuvas_perc', 'queda_rendimento_perc', 'altitude']

In [ ]:
# Geração de estatísticas descritivas / Generating descriptive statistics
print("\n Estatística Descritivas das Variáveis Chave' \n")
format_dict = {
    'reducao_chuvas_perc': '{:.2f}%',
    'queda_rendimento_perc': '{:.2f}%',
    'rendimento_kg_ha': '{:.2f}',
    'altitude': '{:.0f}'
}
display(df[colunas_chave].describe().style.format(format_dict).set_properties(**{'text-align': 'center'}))


In [ ]:
# Análise de Correlação de Pearson / Pearson Correlation Analysis
print('\n Análise de Correlação (Fator Crítico vs. Queda de Rendimento) \n')
correlacao = df[['reducao_chuvas_perc', 'queda_rendimento_perc']].corr()
display(correlacao.style.format('{:.3f}').set_properties(**{'text-align': 'center'}))

In [ ]:
# Visualização Geoespacial Interativa / Interactive Geospatial Visualization
print('\n Mapa Interativo: Queda de Rendimento por Localização\n')

fig = px.scatter_mapbox(
    df,
    lat='latitude',
    lon='longitude',
    color='queda_rendimento_perc',
    size='rendimento_kg_ha',
    hover_name='municipio',
    hover_data={
        'rendimento_kg_ha': ':.2f} kg/ha',
        'queda_rendimento_perc': ':.2f}%',
        'reducao_chuvas_perc': ':.2f}%',
        'latitude': False,
        'longitude': False
    },
    title='Impacto da seca (Queda de Rendimento %) em Cultivo de Soja (Paraná)',
    color_continuous_scale=px.colors.sequential.Reds,
    zoom=6.5,
    height=600
    )
fig.update_layout(mapbox_style='open-street-map')
fig.update_layout(margin={'r':0, 't':50, 'l':0, 'b':0})
fig.show()